<div style="text-align: center;">
<img src="images/stust.png" alt="STUST" class="center" style="width: 900px;"/>
</div>

<hr style="border:4px solid gray"> </hr>

<div style="text-align: center;">    
<br>    
    
# Traffic Light Device with Remote Control
# MQTT + I2C

</div>

<br>
<hr style="border:4px solid gray"> </hr>


<hr style="border:2px solid orange"> </hr>

# [Use MQTT with Raspberry Pi](http://emqx.io/blog/use-mqtt-with-raspberry-pi)

### Install MQTT client library `paho-mqtt`
> Source: https://www.emqx.io/blog/use-mqtt-with-raspberry-pi

* Use the source code to install
>```bash
git clone https://github.com/eclipse/paho.mqtt.python 
cd paho.mqtt.python 
python3 setup.py install
>```

* Use pip3 to install
>```
pip3 install paho-mqtt
>```

### Free public MQTT broker provided by EMQ X:

>```text
Broker         : broker.emqx.io
TCP Port       : 1883
Websocket Port : 8083
>```

# [paho-mqtt Project Description](https://pypi.org/project/paho-mqtt/)
> Source: https://pypi.org/project/paho-mqtt/

<hr style="border:2px solid orange"> </hr>

# Control Arduino-UNO's LEDs via I2C

## Arduino Sketch
<hr style="border:0.5px solid gray">

```C++
#include <Wire.h>

#define I2C_ADDR 11
#define R1 11
#define Y1 12
#define G1 13
#define R2 5
#define Y2 6
#define G2 7


#define MAP 55
#define LOOP 66
#define FOREVER 77

const unsigned LEDS[] = {R1,Y1,G1,R2,Y2,G2};

unsigned len;
byte msg[20];
byte cmd = 0;
unsigned index = 0;
unsigned cntdown = 0;

// -----------------------------------------------------
void setup() {
  Wire.begin(I2C_ADDR);
  Serial.begin(9600);
  for(int i=0; i<6; i++)
    pinMode(LEDS[i],OUTPUT); 
  Serial.print("MQTT + I2C Homework 01: ");
  Serial.println(I2C_ADDR);
  Serial.println("======================================");
  delay(1000);    
  // register onReceive event services           
  Wire.onReceive(irsReceive);
}

void disp_msg() {
    Serial.print("Received data  : ");
    for(int i=0; i<len; i++) {
      Serial.print(msg[i]);
      Serial.print(" ");
    }
    Serial.println();  
}

void mapLED(byte dat) {
  for(int i=0; i<6; i++)
    digitalWrite(LEDS[i],dat&(0b00000001<<i)?HIGH:LOW);
}

// -----------------------------------------------------
void loop() {
  switch(cmd) {
    case 0: break;
    case MAP: 
        mapLED(msg[1]);
        cmd = 0;
      break;
    case LOOP: if (cntdown<=0) {
                cmd=0;
                break;
               }
    case FOREVER:  
        mapLED(msg[index++]);
        delay(msg[index++]*100);
        if (index>=len) {
          index = 1;
          cntdown--;
        }
      break;
    default: break;
  }
  delay(1);
}

// -----------------------------------------------------
void irsReceive(int nb) {
  len = 0;
  while (Wire.available())
    msg[len++] = Wire.read();
  cmd = msg[0];  
  disp_msg();
  index = 0;
  cntdown = 0;
  switch(cmd){
    case LOOP: cntdown = msg[len-1];
               len--;
    case FOREVER: index = 1;
  }
}
```

<hr style="border:2px solid orange"> </hr>

# Control Arduino-UNO's LEDs via I2C

## Raspberry Pi - Python
<hr style="border:0.5px solid gray">


In [63]:
import smbus2 as smbus 
from time import sleep
from random import randrange, random

I2C_ADDR = 11
PORT = 1
I2C = smbus.SMBus(PORT)

# LED Pin Mapped
R1, Y1, G1 = 1, 2, 4
R2, Y2, G2 = 8, 16, 32

MAP = 55
LOOP = 66
FOREVER = 77

# send LED-Control command to Arduino with parameters
# cmd : commands
# dat : additional parameters
def toArduino(cmd, dat=[]):
    global I2C
    bytes_msg = bytes(dat)
    print(f"{cmd}",end=" ")
    I2C.write_i2c_block_data(I2C_ADDR, cmd, bytes_msg)
    #sleep(0.1)

In [76]:
toArduino(FOREVER,[R1|G2,50,R1|Y1|Y2,5,G1|R2,50,Y1|R2|Y2,5])

77 

In [77]:
toArduino(LOOP,[0xff,5,0,5,5])

66 

In [72]:
toArduino(MAP,[7])

55 

<hr style="border:2px solid orange"> </hr>

# Control Arduino-UNO's LEDs via MQTT

## Raspberry Pi - Python

In [78]:
import paho.mqtt.client as mqtt
import threading, os
from time import sleep

TOPIC  = "P203-UNO"
BROKER = "broker.emqx.io"
PORT   = 1883
Client = None

# MQTT-Connected callback function 
def on_connect(client, userdata, flags, rc):
    global TOPIC
    print("Connected with result code "+str(rc))

# MQTT message retrieval callback function for the subscribed topics
def on_message(client, userdata, msg):
    topic = msg.topic
    payload = msg.payload.decode('utf-8')
    print(f"TOPIC({topic}) : MSG({payload})")
    cmds = payload.split(',')
    #print(cmds)
    try:
        if cmds[0]=='EXIT':
            client.disconnect()
        elif cmds[0]=='LOOP':
            cmd = LOOP
            par = [int(c) for c in cmds[1:]]
            toArduino(cmd, par)
        elif cmds[0]=='FOREVER':
            cmd = FOREVER
            par = [int(c) for c in cmds[1:]]
            toArduino(cmd, par)
        elif cmds[0]=='MAP':
            cmd = MAP
            par = [int(cmds[1])]
            toArduino(cmd, par)
    except:
        print("Something wrong!")
            
def connect_mqtt():
    global Client
    # Create MQTT client and connect to the broker    
    Client = mqtt.Client()
    Client.on_connect = on_connect
    Client.on_message = on_message
    # client.username_pw_set("user-name","password")
    Client.connect(BROKER, PORT, keepalive=60)
    # Clear the retained message send by Kittenblock
    Client.publish(topic=TOPIC, payload=None, retain=True)
    # Subscribe interested topic(s)
    Ｃlient.subscribe(TOPIC)

# Main Loop
def main():
    global Client
    Client.loop_forever()

if __name__ == "__main__":
    connect_mqtt()
    main()

Connected with result code 0
TOPIC(P203-UNO) : MSG(EXIT)


<hr style="border:0.5px solid gray"> 

<br><hr style="border:3px solid red"> </hr>
<div style="text-align: left;">         
    
# *Final Project - I2C Traffic-Light Station with MQTT Remote*

</div>
<hr style="border:3px solid red"> </hr>

## Project Description

* This project requires 2 different development boards: `Raspberry Pi` and `Arduino-UNO`.
* Arduino-UNO `I2C Traffic-Light Station` handles the following commands:
    * *LED SET 1: (time unit : 0.1sec, e.g. td-p=10 => 1sec)*
        * `1`  : RED Led off
        * `2`  : RED Led on
        * `3`  : YELLOW Led off
        * `4`  : YELLOW Led on
        * `5`  : GREEN Led off
        * `6`  : GREEN Led on
        * `11, [td-p]` : RED Led blink (period `td-p`, 50% duty-cycle) 
        * `12, [td-p]` : YELLOW Led blink (period `td-p`, 50% duty-cycle)
        * `21, [td-r, td-ry, td-g, td-y]` : RED-to-GREEN sequence
            * R(`td-r`) -> R+Y(`td-ry`) -> G(`td-g`) -> Y(`td-y`)
        * `22, [td-r, td-ry, td-g, td-y]` : GREEN-to-RED sequence
            * G(`td-g`) -> Y(`td-y`) -> R(`td-r`) -> R+Y(`td-ry`) 

    * *LED SET 2: (refer to LED SET 1, all `command code + 100`)*
        * `101`  : RED Led off
        * `102`  : RED Led on
        * ...

* Raspberry Pi `MQTT Remote Control` handles the following tasks:
    * Perform self-test at startup
    * Provide the minimum command set corresponding to your I2C Traffic-Light Station
    * MQTT command format:
        * simple command: `1`, `2`, `3`, `101`, `102`, ...
        * command with paramaters: `11/10`, `21/50/5/60/5`, ...

## Grading rules:
* `50 points`  : Complete command 1 ~ 6 of LED SET 1 
* `60 points`  : Complete command 1 ~ 6 of LED SET 1 and 2 
* `70 points`  : Complete command 1 ~ 12 of LED SET 1 
* `80 points`  : Complete command 1 ~ 12 of LED SET 1 and 2 
* `90 points`  : Complete all commands of LED SET 1 
* `100 points` : Complete all commands of LED SET 1 and 2 
* *You must complete all required programs (C/C++ and Python) for I2C Traffic-Light Station and MQTT Remote Control to get full points.*



<hr style="border:2px solid orange"> </hr>
<br>

<div style="text-align: left;">
<img src="images/break-yang-tr.png" alt="Break" class="center" style="width: 600px;"/>
</div>
